# HuggingFace 模块解读

[视频链接](https://www.bilibili.com/video/BV1Dh411c7BQ?p=2&vd_source=0a8068a684645bf59ae78e7f2c9dbcde)

## NLP 要解决的任务

1. 处理文本数据，首先对文本数据进行分词操作(分词的方法可能会不同，中文常见的就是分词或者分字。
2. 分完的词它不还是字符嘛，计算机还不认识，最终我们希望把这些字符映射成实际的特征(向量)。
3. 输入搞定好了之后，接下来咱们要构建模型了(一般都用预训练模型，例如 BERT,GPT 系列等)。
4. 怎么去完成我们自己的任务呢，基本上就是在预训练模型的基础上进行微调 (训练我们自己数据的过程)


## 基本流程

![process](./img/process.png)


### Tokenizer 分词器

Tokenizer 要做的事:

* 分词，分字以及特殊字符(起始，终止，间隔，分类等特殊字符可以自己设计的)
* 对每一个 token 映射得到一个 ID (每个词都会对应一个唯一的 ID)
* 还有一些辅助信息也可以得到，比如当前词属于哪个句子(还有一些 MASK，表示是否事原来的词还是特殊字符等)


In [ ]:
# %pip install transformers
# %pip install torch

In [2]:
from transformers import AutoTokenizer  # 自动判断使用哪个tokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"  # 根据模型名字来自动加载
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [3]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs,
                   padding=True,
                   truncation=True,
                   return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


In [5]:
tokenizer.decode(inputs["input_ids"][0])

"[CLS] i've been waiting for a huggingface course my whole life. [SEP]"

> 所有的经过分词器编码后的文本都是以 ID 的形式存在的，这样计算机才能认识，才能进行后续的操作。
>
> 分词器编码的结果包括： `input_ids` , `attention_mask`


### 加载模型


In [6]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
model

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

#### 模型输出

Transformer 模块输出的向量通常很大。它一般具有三个维度：

* 批量大小(Batch size)：一次处理的序列数（在我们的示例中为 `2`，就是一次处理两个句子）。
* 序列长度(Sequence length:)：序列的数字表示的长度（在我们的示例中为 `16`，就是句子转换为`input id`后的长度）。
* 隐藏大小(Hidden size)：每个模型输入的向量维度，把 token 都转换成向量。隐藏大小可以非常大（对于较小的模型，768 很常见，在较大的模型中，可以达到 3072 或更多）


In [8]:
# **kwargs 的含义是将字典扩展为关键字参数进行传递，如果 kwargs 等于 {'a':1,'b':2,'c':3} ，那这个代码就等价于 test(a=1,b=2,c=3)
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


## 模型基本逻辑

### 模型头

模型头将隐藏状态的高维向量作为输入，并将其投影到不同的维度。它们通常由一个或几个线性层组成：

![](./img/002.png)

在此图中，模型由其嵌入层和后续层表示。嵌入层将标记化输入中的每个输入 ID 转换为表示关联标记(token)的向量。后续层使用注意机制操纵这些向量，以生成句子的最终表示。

🤗 Transformers中有许多不同的体系结构，每种体系结构都是围绕处理特定任务而设计的。以下是一个非详尽的列表：

*   `*Model` (retrieve the hidden states)
*   `*ForCausalLM`
*   `*ForMaskedLM`
*   `*ForMultipleChoice`
*   `*ForQuestionAnswering`
*   `*ForSequenceClassification`
*   `*ForTokenClassification`
*   以及其他 🤗

对于我们的示例，我们需要一个带有序列分类头的模型（能够将句子分类为肯定或否定）。因此，我们实际上不会使用 `AutoModel` 类，而是使用 `AutoModelForSequenceClassification` ：


In [4]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [5]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

现在，如果我们观察输出的形状，维度将低得多：模型头将我们之前看到的高维向量作为输入，并输出包含两个值的向量（每个标签一个）：


In [6]:
print(outputs.logits.shape)

torch.Size([2, 2])


因为我们只有两个句子和两个标签，所以我们从模型中得到的结果是2 x 2的形状。


### 输出后处理

我们从模型中得到的输出值本身并不一定有意义。我们来看看, 


In [7]:
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


我们的模型预测第一句为 `[-1.5607, 1.6123]` ，第二句为 `[ 4.1692, -3.3464]` 。这些不是概率，而是 _logits_ ，即模型最后一层输出的原始非标准化分数。要转换为概率，它们需要经过[SoftMax](https://en.wikipedia.org/wiki/Softmax_function)层（所有🤗Transformers模型输出logits，因为用于训练的损耗函数通常会将最后的激活函数（如SoftMax）与实际损耗函数（如交叉熵）融合）：


In [8]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5981e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


现在我们可以看到，模型预测第一句为 `[0.0402, 0.9598]` ，第二句为 `[0.9995, 0.0005]` 。这些是可识别的概率分数。

为了获得每个位置对应的标签，我们可以检查模型配置的 `id2label` 属性（下一节将对此进行详细介绍）：


In [9]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

现在我们可以得出结论，该模型预测了以下几点：

*   第一句：否定：0.0402，肯定：0.9598
*   第二句：否定：0.9995，肯定：0.0005

我们已经成功地复制了管道的三个步骤：使用标记化器进行预处理、通过模型传递输入以及后处理！现在，让我们花一些时间深入了解这些步骤中的每一步。


# 模型基本训练方法

## 数据集与模型
